In [9]:
import nest_asyncio
nest_asyncio.apply()

In [1]:
import sys
sys.path.append("../")

from desci_sense.configs import default_init_parser_config
from desci_sense.shared_functions.dataloaders import scrape_post
from desci_sense.shared_functions.parsers.firebase_api_parser import FirebaseAPIParser, PromptCase

In [2]:
config = default_init_parser_config()

In [3]:
# get a few posts for input
urls = ["https://mastodon.social/@psmaldino@qoto.org/111405098400404613",
        "https://mastodon.social/@UlrikeHahn@fediscience.org/111732713776994953",
        "https://mastodon.social/@ronent/111687038322549430"]
posts = [scrape_post(url) for url in urls]

In [4]:
len(posts)

3

In [5]:
parser = FirebaseAPIParser(config=config)

2024-03-17 12:23:50.718 | INFO     | desci_sense.shared_functions.parsers.firebase_api_parser:set_md_extract_method:134 - Setting metadata extraction method to none...
2024-03-17 12:23:50.719 | INFO     | desci_sense.shared_functions.parsers.firebase_api_parser:__init__:97 - Loading parser model (type=openai/gpt-4)...
/Users/ronentamari/anaconda3/envs/asensebot/lib/python3.11/site-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The class `langchain_community.chat_models.openai.ChatOpenAI` was deprecated in langchain-community 0.0.10 and will be removed in 0.2.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import ChatOpenAI`.
  warn_deprecated(
2024-03-17 12:23:50.962 | INFO     | desci_sense.shared_functions.parsers.firebase_api_parser:set_kw_md_extract_method:138 - Setting keywords metadata extraction method to citoid

In [6]:
input_prompts = [parser.create_semantics_prompt_by_case(post, PromptCase.SINGLE_REF) for post in posts]

In [8]:
inputs = [{"input": prompt} for prompt in input_prompts]

In [7]:
chain = parser.prompt_case_dict[PromptCase.SINGLE_REF]["chain"]
chain

PromptTemplate(input_variables=['input'], template='{input}')
| ChatOpenAI(client=<openai.resources.chat.completions.Completions object at 0x14ebc9250>, async_client=<openai.resources.chat.completions.AsyncCompletions object at 0x14ebd4c10>, model_name='openai/gpt-4', temperature=0.6, openai_api_key='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', openai_api_base='https://openrouter.ai/api/v1', openai_proxy='')
| TagTypeParser(allowed_tags=['call-for-papers', 'endorses', 'disagrees', 'agrees', 'indicates-interest', 'funding', 'watching', 'reading', 'listening', 'default', 'review', 'recommendation', 'question', 'quote', 'discussion', 'event', 'job', 'announce'])

In [10]:
results = await chain.abatch(inputs)

In [11]:
results[0]

{'reasoning': '[Reasoning Steps]\n\nThe post is made by an author named Paul Smaldino who is announcing the release of a new paper. The paper is about a model of a group-structured public good, and is published in Phil Trans B. The author also provides a link to the paper.\n\n[Candidate Tags]\n\n<announce>: The author is announcing the release of a new paper. This is evident from the phrase "New paper out".\n<endorses>: The author endorses the paper as he is one of the authors of the paper.',
 'final_answer': '<announce>, <endorses>',
 'single_tag': ['announce'],
 'multi_tag': ['announce', 'endorses']}